In [ ]:
import os

import numpy as np
import pandas as pd
import torch
import wandb
from datasets import load_dataset
from huggingface_hub import notebook_login
from sklearn.metrics import f1_score, roc_auc_score
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import AutoTokenizer, BertForSequenceClassification, DataCollatorWithPadding

In [ ]:
full_config = {
    "project": "VKR",
    "config": {
        "dataset": "cedr",
        "num_labels": 6,
        "labels": {
            0: "no emotion",
            1: "joy",
            2: "sadness",
            3: "surprise",
            4: "fear",
            5: "anger",
        },
        "model": "cointegrated/rubert-tiny2",
        "tokenizer": "cointegrated/rubert-tiny2",
        "problem_type": "multi_label_classification",
        "batch_size": 64,
        "epochs": 42,
        "lr": 1e-5,
    },
    "name": "rubert-tiny2-cedr",
}
config = full_config["config"]

In [ ]:
def binarize_labels(labels, num_labels):
    return [int(len(labels) == 0)] + [int(i in labels) for i in range(num_labels)]


def label2id(class_labels):
    label2id = {label: i for i, label in enumerate(class_labels)}
    id2label = {i: label for label, i in label2id.items()}
    return label2id, id2label

In [ ]:
def calculate_aucs(y_true, y_pred, num_labels):
    return [roc_auc_score(y_true[:, i], y_pred[:, i]) for i in range(num_labels)]


def calculate_f1_score(y_true, y_pred, average, num_labels):
    return [f1_score(y_true[:, i], y_pred[:, i] > 0.5, average=average) for i in range(num_labels)]


def calculate_metrics(y_true, y_pred, num_labels):
    auc_rocs = calculate_aucs(y_true, y_pred, num_labels)

    f1_scores_micro = calculate_f1_score(y_true, y_pred, "micro", num_labels)

    f1_scores_macro = calculate_f1_score(y_true, y_pred, "macro", num_labels)

    df = pd.DataFrame(
        [auc_rocs, f1_scores_micro, f1_scores_macro],
        columns=config["labels"].values(),
        index=["AUC ROC", "F1 micro", "F1 macro"],
    )
    df["mean"] = df.mean(axis=1)
    df["mean(emotions)"] = df.drop("no emotion", axis=1).mean(axis=1)
    return df

In [ ]:
def predict(model, dataloader):
    with torch.no_grad():
        y_true = []
        y_pred = []
        for batch in dataloader:
            batch = batch.to(model.device)
            output = model(**batch)
            y_true.append(batch.labels)
            y_pred.append(torch.softmax(output.logits, -1))
    return torch.cat(y_true).cpu().numpy(), torch.cat(y_pred).cpu().numpy()

In [ ]:
def train(model, train_dataloader, optimizer, epochs, test_dataloader):
    tq = tqdm(range(epochs))

    for epoch in tq:
        model.train()
        for i, batch in enumerate(train_dataloader):
            optimizer.zero_grad()
            print(batch.keys())
            print(type(batch))
            batch = batch.to(model.device)
            output = model(**batch)
            loss = output.loss
            loss.backward()
            optimizer.step()

        # if epoch % 5 == 0:
        model.eval()

        y_true, y_pred = predict(model, train_dataloader)
        train_auc = np.mean(calculate_aucs(y_true, y_pred, config["num_labels"]))

        y_true, y_pred = predict(model, test_dataloader)
        test_auc = np.mean(calculate_aucs(y_true, y_pred, config["num_labels"]))

        tq.set_description(f"loss: {loss.item():4.4f}, AUC: {test_auc:4.4f}")
        wandb.log({"train_auc": train_auc, "test_auc": test_auc, "train_loss": loss.item()})

In [ ]:
dataset = load_dataset(config["dataset"])

Using the latest cached version of the module from /home/seara/.cache/huggingface/modules/datasets_modules/datasets/cedr/117570489cbabbdf8de619bd31918a1cd680a7f286b89d04af340d0691dc2d66 (last modified on Tue Feb 14 12:59:29 2023) since it couldn't be found locally at cedr., or remotely on the Hugging Face Hub.
No config specified, defaulting to: cedr/main
Found cached dataset cedr (/home/seara/.cache/huggingface/datasets/cedr/main/0.1.1/117570489cbabbdf8de619bd31918a1cd680a7f286b89d04af340d0691dc2d66)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config["tokenizer"])

In [ ]:
config["label2id"], config["id2label"] = label2id(config["labels"].values())
processed_dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True), batched=True).map(
    lambda x: {"label": [float(y) for y in binarize_labels(x["labels"], config["num_labels"] - 1)]},
    batched=False,
    remove_columns=["text", "labels", "source"],
)

Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/cedr/main/0.1.1/117570489cbabbdf8de619bd31918a1cd680a7f286b89d04af340d0691dc2d66/cache-cf7aa35d4433e92a.arrow
Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/cedr/main/0.1.1/117570489cbabbdf8de619bd31918a1cd680a7f286b89d04af340d0691dc2d66/cache-c0f8df935201b049.arrow
Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/cedr/main/0.1.1/117570489cbabbdf8de619bd31918a1cd680a7f286b89d04af340d0691dc2d66/cache-6f9f796a9e6674ba.arrow
Loading cached processed dataset at /home/seara/.cache/huggingface/datasets/cedr/main/0.1.1/117570489cbabbdf8de619bd31918a1cd680a7f286b89d04af340d0691dc2d66/cache-ecd0dfddc7f3b5ff.arrow


In [ ]:
model = BertForSequenceClassification.from_pretrained(
    config["model"],
    num_labels=config["num_labels"],
    problem_type=config["problem_type"],
    label2id=config["label2id"],
    id2label=config["id2label"],
)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

train_dataloader = DataLoader(
    processed_dataset["train"],
    batch_size=config["batch_size"],
    drop_last=False,
    shuffle=True,
    num_workers=0,
    collate_fn=data_collator,
)
test_dataloader = DataLoader(
    processed_dataset["test"],
    batch_size=config["batch_size"],
    drop_last=False,
    shuffle=True,
    num_workers=0,
    collate_fn=data_collator,
)

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=config["lr"])

In [ ]:
os.environ["WANDB_NOTEBOOK_NAME"] = "/home/seara/Desktop/Github/vkr/new_era/rubert-tiny2-cedr.ipynb"
wandb.login()
wandb.init(**full_config)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find /home/seara/Desktop/Github/vkr/new_era/rubert-tiny2-cedr.ipynb.
wandb: Currently logged in as: seara. Use `wandb login --relogin` to force relogin


In [ ]:
model.cuda()
train(model, train_dataloader, optimizer, config["epochs"], test_dataloader)

  0%|          | 0/42 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
dict_k

In [ ]:
# notebook_login()

In [ ]:
calculate_metrics(*predict(model, test_dataloader), config["num_labels"]).round(4)

,no emotion,joy,sadness,surprise,fear,anger,mean,mean(emotions)
AUC ROC,0.9239,0.9550,0.9537,0.9045,0.9016,0.7682,0.9011,0.8973
F1 micro,0.8608,0.9400,0.9336,0.9485,0.9559,0.9192,0.9263,0.9372
F1 macro,0.8527,0.9005,0.8975,0.8402,0.8350,0.6768,0.8338,0.8306


In [ ]:
model.push_to_hub(full_config["name"])
tokenizer.push_to_hub(full_config["name"])

CommitInfo(commit_url='https://huggingface.co/seara/rubert-tiny2-cedr/commit/241abd5e1685e91cee1f1843d99cf16c0a8e285f', commit_message='Upload tokenizer', commit_description='', oid='241abd5e1685e91cee1f1843d99cf16c0a8e285f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
wandb.finish()

test_auc,▁▂▂▄▅▅▆▇▇▇▇▇▇▇██████████████████████████
train_auc,▁▂▂▃▄▄▅▆▆▆▆▇▇▇▇▇▇▇▇█████████████████████
train_loss,█▇▆▆▅▄▅▅▅▄▄▄▃▃▃▂▃▃▃▃▂▂▂▂▂▂▃▂▁▂▂▁▁▂▁▁▁▁▁▁
test_auc,0.90113
train_auc,0.99792
train_loss,0.02599
